In [52]:
!pip install --upgrade yt_dlp
!pip install pytubefix"

zsh:1: unmatched "


In [53]:
from pytubefix import YouTube
import os

yt = YouTube("https://www.youtube.com/watch?v=sV2t3tW_JTQ")

video = yt.streams.filter(only_audio=True).first()

print("Enter the destination (leave blank for current directory)")
destination = ""

out_file = video.download(output_path=destination)

base, ext = os.path.splitext(out_file)
new_file = "song" + '.wav'
os.rename(out_file, new_file)

print(yt.title + " has been successfully downloaded.")

Enter the destination (leave blank for current directory)
21 Savage - Bank Account (Official Audio) has been successfully downloaded.


In [54]:
import os
import multiprocessing
import warnings
import numpy as np
from scipy import stats
import pandas as pd
import librosa
from tqdm import tqdm
def columns():
    feature_sizes = dict(chroma_stft=12, chroma_cqt=12, chroma_cens=12,
                         tonnetz=6, mfcc=20, rmse=1, zcr=1,
                         spectral_centroid=1, spectral_bandwidth=1,
                         spectral_contrast=7, spectral_rolloff=1,mel_spec=128)
    moments = ('mean', 'std', 'skew', 'kurtosis', 'median', 'min', 'max')

    columns = []
    for name, size in feature_sizes.items():
        for moment in moments:
            it = ((name, moment, '{:02d}'.format(i+1)) for i in range(size))
            columns.extend(it)

    names = ('feature', 'statistics', 'number')
    columns = pd.MultiIndex.from_tuples(columns, names=names)

    # More efficient to slice if indexes are sorted.
    return columns.sort_values()


def compute_features():

    features = pd.Series(index=columns(), dtype=np.float32)

    # Catch warnings as exceptions (audioread leaks file descriptors).
    warnings.filterwarnings('error', module='librosa')

    def feature_stats(name, values):
        features[name, 'mean'] = np.mean(values, axis=1)
        features[name, 'std'] = np.std(values, axis=1)
        features[name, 'skew'] = stats.skew(values, axis=1)
        features[name, 'kurtosis'] = stats.kurtosis(values, axis=1)
        features[name, 'median'] = np.median(values, axis=1)
        features[name, 'min'] = np.min(values, axis=1)
        features[name, 'max'] = np.max(values, axis=1)

    try:
        x, sr = librosa.load("song.wav", sr=None, mono=True)  # kaiser_fast

        f = librosa.feature.zero_crossing_rate(x, frame_length=2048, hop_length=512)
        feature_stats('zcr', f)

        cqt = np.abs(librosa.cqt(x, sr=sr, hop_length=512, bins_per_octave=12,
                                 n_bins=7*12, tuning=None))
        
        assert cqt.shape[0] == 7 * 12
        assert np.ceil(len(x)/512) <= cqt.shape[1] <= np.ceil(len(x)/512)+1

        f = librosa.feature.chroma_cqt(C=cqt, n_chroma=12, n_octaves=7)
        feature_stats('chroma_cqt', f)
        f = librosa.feature.chroma_cens(C=cqt, n_chroma=12, n_octaves=7)
        feature_stats('chroma_cens', f)
        f = librosa.feature.tonnetz(chroma=f)
        feature_stats('tonnetz', f)

        del cqt
        stft = np.abs(librosa.stft(x, n_fft=2048, hop_length=512))
        assert stft.shape[0] == 1 + 2048 // 2
        assert np.ceil(len(x)/512) <= stft.shape[1] <= np.ceil(len(x)/512)+1
        del x

        f = librosa.feature.chroma_stft(S=stft**2, n_chroma=12)
        feature_stats('chroma_stft', f)

        f = librosa.feature.rms(S=stft)
        feature_stats('rmse', f)

        f = librosa.feature.spectral_centroid(S=stft)
        feature_stats('spectral_centroid', f)
        f = librosa.feature.spectral_bandwidth(S=stft)
        feature_stats('spectral_bandwidth', f)
        f = librosa.feature.spectral_contrast(S=stft, n_bands=6)
        feature_stats('spectral_contrast', f)
        f = librosa.feature.spectral_rolloff(S=stft)
        feature_stats('spectral_rolloff', f)
        f = librosa.feature.melspectrogram(S=stft)
        feature_stats('mel_spec', f)
        
        
        
        mel = librosa.feature.melspectrogram(sr=sr, S=stft**2)
        del stft
        f = librosa.feature.mfcc(S=librosa.power_to_db(mel), n_mfcc=20)
        feature_stats('mfcc', f)
        print(f)

    except Exception as e:
        print('{}'.format( repr(e)))
    os.remove("./song.wav")
    return features

In [58]:
features = pd.DataFrame(compute_features())

/var/folders/gv/571nb6gs4h58lv0gw0__gy280000gn/T/ipykernel_72657/1526570064.py:46: UserWarning: PySoundFile failed. Trying audioread instead.
  x, sr = librosa.load("song.wav", sr=None, mono=True)  # kaiser_fast
/var/folders/gv/571nb6gs4h58lv0gw0__gy280000gn/T/ipykernel_72657/1526570064.py:37: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.02900229]' has dtype incompatible with float32, please explicitly cast to a compatible dtype first.
  features[name, 'mean'] = np.mean(values, axis=1)


[[-458.8114 -458.8114 -458.8114 ... -458.8114 -458.8114 -458.8114]
 [   0.        0.        0.     ...    0.        0.        0.    ]
 [   0.        0.        0.     ...    0.        0.        0.    ]
 ...
 [   0.        0.        0.     ...    0.        0.        0.    ]
 [   0.        0.        0.     ...    0.        0.        0.    ]
 [   0.        0.        0.     ...    0.        0.        0.    ]]


In [60]:
features = features.transpose()

In [62]:
mel_spec = features['mel_spec']

In [64]:
mel_spec.columns = pd.MultiIndex.from_tuples([
    ('mel_spec',) + col if isinstance(col, tuple) else ('mel_spec', col)
    for col in mel_spec.columns
])

In [66]:
mel_spec

mel_spec                                                                \
   kurtosis                                                                 
         01        02         03        04        05        06         07   
0 -0.976424  5.066773  17.145786  9.009282  8.775974  12.18237  11.746777   

                                   ...                                        \
                                   ...      std                                
         08         09         10  ...       90        91        92       93   
0  7.874873  11.573224  10.567286  ...  0.00052  0.000502  0.000493  0.00048   

                                                               
                                                               
         94        95        96        97        98        99  
0  0.000468  0.000465  0.000448  0.000448  0.000439  0.000441  

[1 rows x 896 columns]

In [68]:
features = features.drop('mel_spec',axis=1).merge(mel_spec,left_index=True,right_index=True)

In [81]:
features['mel_spec']

kurtosis                                                                \
         01        02         03        04        05        06         07   
0 -0.976424  5.066773  17.145786  9.009282  8.775974  12.18237  11.746777   

                                   ...      std                               \
         08         09         10  ...       90        91        92       93   
0  7.874873  11.573224  10.567286  ...  0.00052  0.000502  0.000493  0.00048   

                                                               
         94        95        96        97        98        99  
0  0.000468  0.000465  0.000448  0.000448  0.000439  0.000441  

[1 rows x 896 columns]

In [84]:
features.to_csv("song.csv")


In [74]:
import pickle
with open('final_optimized_classification_model.pkl', 'rb') as f:
    model = pickle.load(f)

FileNotFoundError: [Errno 2] No such file or directory: 'final_optimized_classification_model.pkl'

In [372]:
model.predict_proba(features)

array([15])